Lambda School Data Science

*Unit 2, Sprint 1, Module 4*

---

# Logistic Regression


## Assignment 🌯

You'll use a [**dataset of 400+ burrito reviews**](https://srcole.github.io/100burritos/). How accurately can you predict whether a burrito is rated 'Great'?

> We have developed a 10-dimensional system for rating the burritos in San Diego. ... Generate models for what makes a burrito great and investigate correlations in its dimensions.

- [ ] Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.
- [ ] Begin with baselines for classification.
- [ ] Use scikit-learn for logistic regression.
- [ ] Get your model's validation accuracy. (Multiple times if you try multiple iterations.)
- [ ] Get your model's test accuracy. (One time, at the end.)
- [ ] Commit your notebook to your fork of the GitHub repo.


## Stretch Goals

- [ ] Add your own stretch goal(s) !
- [ ] Make exploratory visualizations.
- [ ] Do one-hot encoding.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Get and plot your coefficients.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [164]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [165]:
# Load data downloaded from https://srcole.github.io/100burritos/
import pandas as pd
df = pd.read_csv(DATA_PATH+'burritos/burritos.csv')

In [166]:
# Derive binary classification target:
# We define a 'Great' burrito as having an
# overall rating of 4 or higher, on a 5 point scale.
# Drop unrated burritos.
df = df.dropna(subset=['overall'])
df['Great'] = df['overall'] >= 4

In [167]:
# Clean/combine the Burrito categories
df['Burrito'] = df['Burrito'].str.lower()

california = df['Burrito'].str.contains('california')
asada = df['Burrito'].str.contains('asada')
surf = df['Burrito'].str.contains('surf')
carnitas = df['Burrito'].str.contains('carnitas')

df.loc[california, 'Burrito'] = 'California'
df.loc[asada, 'Burrito'] = 'Asada'
df.loc[surf, 'Burrito'] = 'Surf & Turf'
df.loc[carnitas, 'Burrito'] = 'Carnitas'
df.loc[~california & ~asada & ~surf & ~carnitas, 'Burrito'] = 'Other'

In [168]:
# Drop some high cardinality categoricals
df = df.drop(columns=['Notes', 'Location', 'Reviewer', 'Address', 'URL', 'Neighborhood'])

In [169]:
# Drop some columns to prevent "leakage"
df = df.drop(columns=['Rec', 'overall'])

In [170]:
df.head()

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,...,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
0,California,1/18/2016,3.5,4.2,NaN,6.49,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,California,1/24/2016,3.5,3.3,NaN,5.45,3.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,Carnitas,1/24/2016,NaN,NaN,NaN,4.85,1.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,Asada,1/24/2016,NaN,NaN,NaN,5.25,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,California,1/27/2016,4.0,3.8,x,6.59,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [171]:
df['Date'] = pd.to_datetime(df['Date'],infer_datetime_format=True)

In [172]:
df.head(1)

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,...,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
0,California,2016-01-18,3.5,4.2,NaN,6.49,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [173]:
cutoff_2017 = pd.to_datetime('01/01/2017')
cutoff_2018 = pd.to_datetime('01/01/2018')
train = df[df.Date < cutoff_2017]
val = df[(df.Date >= cutoff_2017) & (df.Date < cutoff_2018)]
test = df[df.Date >= cutoff_2018]

In [174]:
train.columns

Index(['Burrito', 'Date', 'Yelp', 'Google', 'Chips', 'Cost', 'Hunger',
       'Mass (g)', 'Density (g/mL)', 'Length', 'Circum', 'Volume', 'Tortilla',
       'Temp', 'Meat', 'Fillings', 'Meat:filling', 'Uniformity', 'Salsa',
       'Synergy', 'Wrap', 'Unreliable', 'NonSD', 'Beef', 'Pico', 'Guac',
       'Cheese', 'Fries', 'Sour cream', 'Pork', 'Chicken', 'Shrimp', 'Fish',
       'Rice', 'Beans', 'Lettuce', 'Tomato', 'Bell peper', 'Carrots',
       'Cabbage', 'Sauce', 'Salsa.1', 'Cilantro', 'Onion', 'Taquito',
       'Pineapple', 'Ham', 'Chile relleno', 'Nopales', 'Lobster', 'Queso',
       'Egg', 'Mushroom', 'Bacon', 'Sushi', 'Avocado', 'Corn', 'Zucchini',
       'Great'],
      dtype='object')

In [175]:
from sklearn.model_selection import train_test_split

X = train.drop(columns=['Great'])
y = train['Great']

X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=.015,stratify=y,random_state=42)
print(X_train.shape,X_val.shape)

(293, 58) (5, 58)


In [176]:
y_train.value_counts(normalize=True)

False    0.590444
True     0.409556
Name: Great, dtype: float64

In [177]:
y_val.value_counts(normalize=True)

False    0.6
True     0.4
Name: Great, dtype: float64

In [178]:
majority_class = y_train.mode()[0]
y_pred = [majority_class]*len(y_train)

In [179]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train, y_pred)

0.590443686006826

In [180]:
y_pred = [majority_class] * len(y_val)
accuracy_score(y_val,y_pred)

0.6

In [181]:
from sklearn.dummy import DummyClassifier

baseline = DummyClassifier(strategy='most_frequent')
baseline.fit(X_train, y_train)

y_pred = baseline.predict(X_val)
accuracy_score(y_val,y_pred)

0.6

In [182]:
from sklearn.linear_model import LinearRegression

linear_reg = LinearRegression()

features = ['Cost','Hunger','Meat','Fillings']
X_val = val[features]
X_train = train[features]
y_train = train['Great']
y_val = val['Great']

from sklearn.impute import SimpleImputer
imputer = SimpleImputer()
X_train_imputed = imputer.fit_transform(X_train)
X_val_imputed = imputer.transform(X_val)

linear_reg.fit(X_train_imputed, y_train)

linear_reg.predict(X_val_imputed)

array([ 0.48939744,  0.51500156,  0.42610554,  0.40620002,  0.06376791,
        0.39997675,  0.69081933,  0.59735883,  0.21483083,  0.5004475 ,
        0.55299895,  0.2338701 ,  0.60601342,  0.39689221,  0.71892427,
        0.53748202,  0.40564414,  0.63471381,  0.69088943,  0.78708388,
        0.59469498,  0.28845205,  0.18524727,  0.62728653,  0.65372491,
        0.71647065,  0.77136978,  0.51008346,  0.6202762 ,  0.70673157,
        0.54122649,  0.39095579,  0.48900475,  0.3928103 ,  0.34356022,
        0.44375171,  0.40879126,  0.41346595,  0.46259536,  0.21704487,
        0.4015745 ,  0.42959214,  0.26416695,  0.62272982,  0.52834098,
        0.49694234, -0.10057454, -0.11453024,  0.60240634, -0.22647555,
        0.28787531,  0.39927572,  0.72621501,  0.63002055,  0.42555927,
        0.18297215,  0.25621102,  0.16006431,  0.38979586,  0.39770214,
       -0.21249319,  0.40039737, -0.02035286,  0.41557149,  0.43482881,
        0.58619963,  0.41388901,  1.0463702 ,  0.42705251,  0.28

In [183]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(solver='lbfgs')
log_reg.fit(X_train_imputed,y_train)
print('Validation Accuracy',log_reg.score(X_val_imputed,y_val))

Validation Accuracy 0.9058823529411765


In [184]:
log_reg.fit(X_val_imputed,y_val)
print('Test Accuracy',log_reg.score(X_val_imputed,y_val))

Test Accuracy 0.8941176470588236


In [185]:
#  Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.
#  Begin with baselines for classification.
#  Use scikit-learn for logistic regression.
#  Get your model's validation accuracy. (Multiple times if you try multiple iterations.)
#  Get your model's test accuracy. (One time, at the end.)
#  Commit your notebook to your fork of the GitHub repo.